<a href="https://colab.research.google.com/github/alirempel/cap-comp215/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import time
from pprint import pprint

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from scipy.signal import correlate2d

# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

In [ ]:
# Qualitative colour map with value 0 set to white
tab20_mod = matplotlib.colormaps['tab20']
tab20_mod.colors = ((1,1,1,1), *tab20_mod.colors[1:])

def plot_2d_array(array, axes=None, title='', cmap=tab20_mod, **options):
    """
    Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue.

    :param axes: the axes to plot on, or None to use the `plt.gca()` (current axes)
    :param options: keyword arguments passed directly to `plt.imshow()`
           see https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html
    """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    # Turn off axes labels and tick marks
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False ,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False,)
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(array, cmap=cmap, **options)

In [ ]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step * self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()  # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)  # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()  # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

In [ ]:
def make_locations(n, m):
    """ Return list of (x, y) coordinates for all locations on n x m grid """
    return [(i, j) for i in range(n) for j in range(m)]

In [ ]:
class DEER:
    #max_vision=6
    max_weight=10
    initial_weight_range=(5, 9)

    def __init__(self, loc=(0,0), weight=None):
        """Creates a new agent at the given location.

        loc: (x,y) tuple coordinate
        params: define agent's attributes - by default these are drawn from uniform distributions defined by class variables
        """
        self.loc = tuple(loc)
        self.weight = weight or np.random.uniform(*self.initial_weight_range)

    @classmethod
    def make_agents(cls, num_agents, n, m):
        """ Factory: return a list Agent objects at random locations in n x m grid """
        # all (x,y) locations in an n x m grid...
        locations = make_locations(n, m)
        assert num_agents <= len(locations)  # verify pre-condition: there are enough locations for all agents
        # randomize the locations and construct the desired number of agents at random locations
        np.random.shuffle(locations)
        return [cls(locations[i]) for i in range(num_agents)]

    def visible_locations(self,x,y):
        """Return a list of (x,y) cell coordinates that are "visible" to the deer """
        neighbours = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        return neighbours

    def find_grass(self, env):
        """Finds a empty cell with the grass sugar within this agent's vision.

        env: the grass area the deer lives in
        returns: tuple, coordinates of random neighbouring cell with grass
        """
        # find all empty, visible cells, "wrapping" vision around edges of environment
        visible_locs = env.wrap_locations( self.visible_locations(*self.loc) )
        empty_locs = env.get_empty_locations(visible_locs)

        # return the cell with the highest sugar content from empty, visible cells...
        if len(empty_locs) > 0:
            i = np.argmax(env.get_sugar(empty_locs)) # (in case of tie, argmax returns the first, which is the closest)
            return empty_locs[i]
        else:  # there are no empty visible cells, so only choice is to stay put
            return self.loc

    def step(self, env):
        """Look around, move, and harvest.

        env: Sugarscape
        """
        self.loc = self.find_highest_value_cell(env)
        self.sugar += env.harvest(self.loc) - self.metabolism

    def is_starving(self):
        """Checks if sugar has gone negative."""
        return self.weight <= 0

In [ ]:
class DeerModel:
    """ 2D Cellular Automaton that simulates a deer-populated landscape """

    EMPTY = 0
    OCCUPIED = 1

    '''
    # Define a colour map that maps each cell state to an intuitive colour.
    cmap = [(1, 0.5, 0), (0, 1, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (1, 0, 0)]
    cmap[EMPTY] = (1, 0.5, 0)  # brown
    cmap[OCCUPIED] = (0, 1, 0)  # green
    cmap[DEER] = (1, 0, 0)  # red
    forest_colour_map = matplotlib.colors.ListedColormap(cmap)
    '''
    # TODO: define a sensible correlation kernel to detect cardinal neighbourhood on fire
    kernel = np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]])


    def __init__(self, n, p=0.01, q=0.5, M=20):
        """Initializes the model.

        n: number of rows
        p: probability an empty cells grows grass (becomes occupied)
        q: initial grass density (probability cell has grass (is occupied) in initial state)
        M: max weight for deer
        """
        self.p = p
        self.M = M
        # initialize landscape with approx. q proportion of cells OCCUPIED
        self.state = np.random.choice([self.OCCUPIED, self.EMPTY], (n, n), p=[q, 1 - q])

    def add_deer(self,positions = [(0,0),(1,1)]):
      '''Adds deer to landscape in positions specified.

      positions: list of tuples containing row,column coords for deer to be placed
      '''
      for row,col in positions:
        self.state[row][col] = self.M


    def step(self):
        """Executes one time step, applying the CA rules to regenerate grass, move deer, and control grazing."""

        is_empty = self.state == self.EMPTY
        is_occupied = self.state == self.OCCUPIED
        has_deer = self.state > 10


        self.state[is_empty] = np.random.choice([self.OCCUPIED,self.EMPTY], len(self.state[is_empty]), 1, (self.p, 1-self.p))
        self.state[has_deer] = self.EMPTY #if deer_weight != M


    def num_occupied(self):
        """ return the number of cells occupied by grass """
        return len(self.state[self.state == self.OCCUPIED])


    def pct_occupied(self):
        """ return the proportion of cells occupied by grass """
        return self.num_occupied() / self.state.size


    def draw(self, axes=None, step=''):
        """Draws the CA cells using the forest colour map so values are coloured intuitively."""
        axes = axes or plt.gca()
        title = f'Time:{step} Occupied: {round(self.pct_occupied() * 100, 2)}%'
        plot_2d_array(self.state, axes=axes, title=title)
                      #cmap=self.forest_colour_map, vmin=0, vmax=len(self.forest_colour_map.colors))

In [ ]:
deer = DeerModel(10)
deer.add_deer()
deer_an = Animation2D(model=deer)
deer_an.animate(interval=.1)

AttributeError: 'DeerModel' object has no attribute 'DEER'

In [ ]:
class Sugarscape(Cell2D):
    """Represents an Epstein-Axtell Sugarscape."""

    def __init__(self, n, agents, grow_rate=1, replace_agents=False):
        """Initializes the attributes.

        n: number of rows and columns
        agents: iterable of agents, with random locations on (n, n)
        replace_agents: Agent model to use to replace dead agents with, or False for no replacement
        grow_rate: sugar re-growth rate
        """
        assert(len(agents) <= n**2)  # can't have more agents than there are grid cells

        self.n = n
        self.agents = agents
        self.grow_rate = grow_rate
        self.replace_agents = replace_agents

        # make the capacity array  (constant - capacity never changes - represents upper bound)
        self.capacity = make_capacity_landscape(n)
        # initially all cells are at capacity, this array is the dynamic state of sugar at each loc
        self.array = self.capacity.copy()

        # keep track of which cells are unoccupied
        self.unoccupied = set(make_locations(n, n)) - set(agent.loc for agent in self.agents)

        # tracking variables
        self.agent_count_seq = []

    def wrap_locations(self, locations):
        """ return listt of (x,y) locations, where each location is "wrapped" so it falls within the sugarscape grid

        locations: iterable of 2-tuple (x,y) locations, some of which may fall outside grid dimensions
        """
        return [(x%self.n, y%self.n) for x,y in locations]

    def get_empty_locations(self, locations):
        """ select and return list locations that are unoccupied """
        return [loc for loc in locations if loc in self.unoccupied]

    def get_sugar(self, locations):
        """ return list of sugar level at each location """
        return [self.array[loc] for loc in locations]

    def grow(self):
        """ add sugar to all cells and caps them by capacity."""
        self.array = np.minimum(self.array + self.grow_rate, self.capacity)

    def harvest(self, loc):
        """ remove and return the sugar from (x, y) `loc` """
        sugar = self.array[loc]
        self.array[loc] = 0
        return sugar

    def step(self):
        """ Execute one time step. """
        # loop through the agents in random order
        for agent in np.random.permutation(self.agents):
            # mark the agent's cell unoccupied and allow them to "step" to new location
            self.unoccupied.add(agent.loc)
            agent.step(self)

            # if the agent is dead, remove from model and potential replace with new agent
            if agent.is_starving() or agent.is_old():
                self.agents.remove(agent)
                if self.replace_agents:
                    self.add_agent(AgentModel=self.replace_agents)
            else:
                # otherwise mark its new cell as occupied
                self.unoccupied.remove(agent.loc)

        # update the time series tracking data
        self.agent_count_seq.append(len(self.agents))

        # grow back some sugar
        self.grow()
        return len(self.agents)

    def add_agent(self, AgentModel):
        """ return a new agent at a random, unoccuped location """
        new_agent = AgentModel( loc=random.sample(self.unoccupied, k=1)[0] )
        self.agents.append(new_agent)
        self.unoccupied.remove(new_agent.loc)
        return new_agent

    def draw(self):
        """Draws the Sugarscape with its agents """
        draw_array(self.array, cmap='YlOrRd', vmax=9, origin='lower')
        if self.agents:
            self._draw_agents()

    def _draw_agents(self):
        """ Draw the agents in centre of cell they occupy """
        # Transform from (col, row) to centre (x, y) of cell coordinate.
        rows, cols = np.transpose([agent.loc for agent in self.agents])
        xs = cols + 0.5
        ys = rows + 0.5
        plt.plot(xs, ys, '.', color='red')